In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

import sympy
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal

In [2]:
Nx, Ny, Nz= symbols('Nx Ny Nz', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
name = Idx('name')
dx, dy, dz = symbols('dx dy dz')

In [3]:
var_names = []
name_alpha = symbols('alpha')
alpha = IndexedBase('state_fab'+str(name_alpha))[i,j,k,str(name_alpha)]

var_names.append(str(name_alpha))

In [4]:
name_betaL = ixp.declarerank1('beta')
betaL = ixp.zerorank1(DIM=3)

for itr in range(len(directions)):
    betaL[itr] = IndexedBase('state_fab'+str(name_betaL[itr]))[i,j,k,str(name_betaL[itr])]

for itr in range(len(name_betaL)):
    var_names.append(str(name_betaL[itr]))

name_betaL = np.array(name_betaL)

In [5]:
name_gLL = ixp.declarerank2('g','sym01')

gLL = ixp.zerorank2(DIM=3)
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gLL[itr1][itr2] = IndexedBase('state_fab'+str(name_gLL[itr1][itr2]))[i,j,k,str(name_gLL[itr1][itr2])]

for itri in range(len(name_gLL)):
    for itrj in range(itri,len(name_gLL)):
        var_names.append(str(name_gLL[itri][itrj]))
        
name_gLL = np.array(name_gLL)

In [6]:
name_KLL = ixp.declarerank2('K','sym01')

KLL = ixp.zerorank2(DIM=3)
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        KLL[itr1][itr2] = IndexedBase('state_fab'+str(name_KLL[itr1][itr2]))[i,j,k,str(name_KLL[itr1][itr2])]

#gamLL = np.array(gamLL)

for itri in range(len(name_KLL)):
    for itrj in range(itri,len(name_KLL)):
        var_names.append(str(name_KLL[itri][itrj]))
        
name_KLL = np.array(name_KLL)

In [7]:
name_gUU = ixp.declarerank2('ginv','sym01')
#for itri in range(len(name_gUU)):
 #   for itrj in range(itri,len(name_gUU)):
  #      var_names.append(str(name_gUU[itri][itrj]))
name_gUU = np.array(name_gUU)

In [8]:
DcgLLL = ixp.zerorank3(3)
GammaULL = ixp.zerorank3(3)

for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            DcgLLL[itri][itrj][itrk] = der(name_gLL[itri][itrj],itrk)
            
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                GammaULL[itrk][itri][itrj] += 1/2*name_gUU[itrk][itrl]*(DcgLLL[itrl][itri][itrj]+DcgLLL[itrl][itrj][itri]-DcgLLL[itri][itrj][itrl])

In [9]:
covbetaLL = ixp.zerorank2(DIM=3)
covbetaTransLL = ixp.zerorank2(DIM=3)
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        covbetaLL[itri][itrj] = der(name_betaL[itri],itrj)
        covbetaTransLL[itri][itrj] = der(name_betaL[itrj],itri)
        for itrk in range(len(directions)):
            covbetaLL[itri][itrj] -= GammaULL[itrk][itri][itrj]*name_betaL[itrk]
            covbetaTransLL[itri][itrj] -= GammaULL[itrk][itrj][itri]*name_betaL[itrk]

In [10]:
g_RHS = -2*name_alpha*name_KLL+covbetaLL+covbetaTransLL

In [12]:
print("amrex::Real " + str(name_alpha) + " = " + AMReXcode(alpha,var_names)+'\n')

for itr in range(len(directions)):
    print("amrex::Real " + str(name_betaL[itr]) + " = " + AMReXcode(betaL[itr],var_names)+'\n')
    
for itri in range(len(directions)):
    for itrj in range(itri,len(directions)):
        print("amrex::Real " + str(name_gLL[itri][itrj]) + " = " + AMReXcode(gLL[itri][itrj],var_names)+'\n')
        
for itri in range(len(directions)):
    for itrj in range(itri,len(directions)):
        print("amrex::Real " + str(name_KLL[itri][itrj]) + " = " + AMReXcode(KLL[itri][itrj],var_names)+'\n')

for itri in range(len(directions)):
    for itrj in range(len(directions)):
        print("amrex::Real " + str(der(name_betaL[itri],itrj)) + " = " + AMReXcode(Dc(betaL[itri],directions[itrj]),var_names)+'\n')
        
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(itrj,len(directions)):
            print("amrex::Real " + str(der(der(name_betaL[itri],itrj),itrk)) + " = " + AMReXcode(sp.simplify(Dc(Dc(betaL[itri],directions[itrj]),directions[itrk])),var_names)+'\n')

for itri in range(len(directions)):
    for itrj in range(itri,len(directions)):
        for itrk in range(len(directions)):
            print("amrex::Real " + str(der(name_gLL[itri][itrj],itrk)) + " = " + AMReXcode(Dc(gLL[itri][itrj],directions[itrk]),var_names)+'\n')
        
for itri in range(len(directions)):
    for itrj in range(itrj,len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(itrk,len(directions)):
                print("amrex::Real " + str(der(der(name_gLL[itri][itrj],itrk),itrl)) + " = " + AMReXcode(sp.simplify(Dc(Dc(gLL[itri][itrj],directions[itrk]),directions[itrl])),var_names)+'\n')
                
for itri in range(len(directions)):
    for itrj in range(itri,len(directions)):
        for itrk in range(len(directions)):
            print("amrex::Real " + str(der(name_KLL[itri][itrj],itrk)) + " = " + AMReXcode(Dc(KLL[itri][itrj],directions[itrk]),var_names)+'\n')
        
for itri in range(len(directions)):
    for itrj in range(itrj,len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(itrk,len(directions)):
                print("amrex::Real " + str(der(der(name_KLL[itri][itrj],itrk),itrl)) + " = " + AMReXcode(sp.simplify(Dc(Dc(KLL[itri][itrj],directions[itrk]),directions[itrl])),var_names)+'\n')
            
for itri in range(len(directions)):
    for itrj in range(itri,len(directions)):
        print(AMReXcode(g_RHS[itri][itrj],declare_rhs=True,rhsname = str(name_gLL[itri][itrj]))+'\n')

amrex::Real alpha = state_fab(i, j, k, Idx::alpha);

amrex::Real beta0 = state_fab(i, j, k, Idx::beta0);

amrex::Real beta1 = state_fab(i, j, k, Idx::beta1);

amrex::Real beta2 = state_fab(i, j, k, Idx::beta2);

amrex::Real g00 = state_fab(i, j, k, Idx::g00);

amrex::Real g01 = state_fab(i, j, k, Idx::g01);

amrex::Real g02 = state_fab(i, j, k, Idx::g02);

amrex::Real g11 = state_fab(i, j, k, Idx::g11);

amrex::Real g12 = state_fab(i, j, k, Idx::g12);

amrex::Real g22 = state_fab(i, j, k, Idx::g22);

amrex::Real K00 = state_fab(i, j, k, Idx::K00);

amrex::Real K01 = state_fab(i, j, k, Idx::K01);

amrex::Real K02 = state_fab(i, j, k, Idx::K02);

amrex::Real K11 = state_fab(i, j, k, Idx::K11);

amrex::Real K12 = state_fab(i, j, k, Idx::K12);

amrex::Real K22 = state_fab(i, j, k, Idx::K22);

amrex::Real dbeta00 = (1.0L/2.0L)*(state_fab(i + 1, j, k, Idx::beta0) - state_fab(i - 1, j, k, Idx::beta0))/dx[0];

amrex::Real dbeta01 = (1.0L/2.0L)*(state_fab(i, j + 1, k, Idx::beta0) - state_fab(i,

amrex::Real ddg0222 = (1.0L/4.0L)*(state_fab(i, j, k + 2, Idx::g02) + state_fab(i, j, k - 2, Idx::g02) - 2*state_fab(i, j, k, Idx::g02))/std::pow(dx[2], 2);

amrex::Real ddg1200 = (1.0L/4.0L)*(state_fab(i + 2, j, k, Idx::g12) + state_fab(i - 2, j, k, Idx::g12) - 2*state_fab(i, j, k, Idx::g12))/std::pow(dx[0], 2);

amrex::Real ddg1201 = (1.0L/4.0L)*(state_fab(i + 1, j + 1, k, Idx::g12) - state_fab(i + 1, j - 1, k, Idx::g12) - state_fab(i - 1, j + 1, k, Idx::g12) + state_fab(i - 1, j - 1, k, Idx::g12))/(dx[0]*dx[1]);

amrex::Real ddg1202 = (1.0L/4.0L)*(state_fab(i + 1, j, k + 1, Idx::g12) - state_fab(i + 1, j, k - 1, Idx::g12) - state_fab(i - 1, j, k + 1, Idx::g12) + state_fab(i - 1, j, k - 1, Idx::g12))/(dx[0]*dx[2]);

amrex::Real ddg1211 = (1.0L/4.0L)*(state_fab(i, j + 2, k, Idx::g12) + state_fab(i, j - 2, k, Idx::g12) - 2*state_fab(i, j, k, Idx::g12))/std::pow(dx[1], 2);

amrex::Real ddg1212 = (1.0L/4.0L)*(state_fab(i, j + 1, k + 1, Idx::g12) - state_fab(i, j + 1, k - 1, Idx::g12) - s